In [41]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
import nltk
from sklearn import ensemble
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

nltk.download('gutenberg')
!python -m spacy download en

[nltk_data] Error loading gutenberg: <urlopen error [Errno 8] nodename
[nltk_data]     nor servname provided, or not known>
Traceback (most recent call last):
  File "/Users/Sri/anaconda3/lib/python3.6/site-packages/urllib3/connection.py", line 141, in _new_conn
    (self.host, self.port), self.timeout, **extra_kw)
  File "/Users/Sri/anaconda3/lib/python3.6/site-packages/urllib3/util/connection.py", line 60, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/Users/Sri/anaconda3/lib/python3.6/socket.py", line 743, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 8] nodename nor servname provided, or not known

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/Sri/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py", line 601, in urlopen
    chunked=chunked)
  File "/Users/Sri/anaconda3/lib

In [42]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
import nltk
from sklearn import ensemble
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.svm import SVC

Supervised NLP requires a pre-labelled dataset for training and testing, and is generally interested in categorizing text in various ways. In this case, we are going to try to predict whether a sentence comes from _Alice in Wonderland_ by Lewis Carroll or _Persuasion_ by Jane Austen. We can use any of the supervised models we've covered previously, as long as they allow categorical outcomes. In this case, we'll try Random Forests, SVM, and KNN.

Our feature-generation approach will be something called _BoW_, or _Bag of Words_. BoW is quite simple: For each sentence, we count how many times each word appears. We will then use those counts as features.

**Note**: Since processing all the text takes around ~5-10 minutes, in the cell below we are taking only the first tenth of each text. If you want to experiment, feel free to change the following code in the next cell:

```python
alice = text_cleaner(alice[:int(len(alice)/10)])
persuasion = text_cleaner(persuasion[:int(len(persuasion)/10)])
```
to 

```python
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)
```

In [311]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = re.sub('(?i)CHAPTER .*', '', text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

alice = text_cleaner(alice[:int(len(alice)/10)])
persuasion = text_cleaner(persuasion[:int(len(persuasion)/10)])

In [313]:
int(len(alice)/10)

1411

In [27]:
alice[0:10]

'Alice was '

In [28]:
persuasion[0:10]

'Sir Walter'

In [400]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [401]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()
sentences.shape

(443, 2)

Time to bag some words!  Since spaCy has already tokenized and labelled our data, we can move directly to recording how often various words occur.  We will exclude stopwords and punctuation.  In addition, in an attempt to keep our feature space from exploding, we will work with lemmas (root words) rather than the raw text terms, and we'll only use the 2000 most common words for each text.

In [181]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 100 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [182]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 100
Processing row 200
Processing row 300
Processing row 400


,Nay,passage,deputation,inhabit,sense,nation,perfectly,steal,elegance,soothing,...,tired,Hall,whisker,embarrassment,tiny,likely,LOVE,great,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll


## Trying out BoW

Now let's give the bag of words features a whirl by trying a random forest.

In [183]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.9773584905660377

Test set score: 0.8426966292134831


/Users/Sri/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Holy overfitting, Batman! Overfitting is a known problem when using bag of words, since it basically involves throwing a massive number of features at a model – some of those features (in this case, word frequencies) will capture noise in the training set. Since overfitting is also a known problem with Random Forests, the divergence between training score and test score is expected.


## BoW with Logistic Regression

Let's try a technique with some protection against overfitting due to extraneous features – logistic regression with ridge regularization (from ridge regression, also called L2 regularization).

In [11]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty='l2') # No need to specify l2 as it's the default. But we put it for demonstration.
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

C:\Users\nagad\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(265, 1610) (265,)
Training set score: 0.969811320754717

Test set score: 0.8651685393258427


Logistic regression performs a bit better than the random forest.  

# BoW with Gradient Boosting

And finally, let's see what gradient boosting can do:

In [184]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.969811320754717

Test set score: 0.8202247191011236


Looks like logistic regression is the winner, but there's room for improvement.

# Same model, new inputs

What if we feed the model a different novel by Jane Austen, like _Emma_?  Will it be able to distinguish Austen from Carroll with the same level of accuracy if we insert a different sample of Austen's writing?

First, we need to process _Emma_ the same way we processed the other data, and combine it with the Alice data. Remember that for computation time concerns, we only took the first tenth of the Alice text. Emma is pretty long. **So in order to get comparable length texts, we take the first sixtieth of Emma**. Again, if you want to experiment, you can take the whole texts of each.

In [13]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma[:int(len(emma)/60)])
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [14]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [15]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

In [16]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 100
done


In [17]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.6979591836734694


col_0,Austen,Carroll
row_0,,
Austen,158,12
Carroll,62,13


Well look at that!  NLP approaches are generally effective on the same type of material as they were trained on. It looks like this model is actually able to differentiate multiple works by Austen from Alice in Wonderland.  Now the question is whether the model is very good at identifying Austen, or very good at identifying Alice in Wonderland, or both...

# Challenge 0:

Recall that the logistic regression model's best performance on the test set was 93%.  See what you can do to improve performance.  Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires.  Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.  

# Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work.  This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.

In [ ]:
#Adding phrases to the features

In [239]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]

def more_bag_of_words(doc):
    
    # Filter out punctuation and stop words.
    phrases = [chunk.text for chunk in doc.noun_chunks
               if (' ' in str(chunk)) and not ('*' in str(chunk))]
    
    # Return the most common words.
    return [item[0] for item in Counter(phrases).most_common(2000)]    

def more_bag_of_words_ent(doc):
    
    # Filter out punctuation and stop words.
    entities = [chunk.label_ for chunk in doc.ents]
    #if (' ' in str(chunk)) and not ('*' in str(chunk))
    # Return the most common words.
    return [item[0] for item in Counter(entities).most_common(2000)]  

def more_bag_of_words_pos(doc):
    
    sentences=doc.sents
    
    pos = [token.pos_ for sent in sentences for token in sent]
    #if (' ' in str(chunk)) and not ('*' in str(chunk))
    # Return the most common words.
    #return [item[0] for item in Counter(pos).most_common(2000)]  
    return pos
    
# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)
alicephrases = more_bag_of_words(alice_doc)
persuasionphrases = more_bag_of_words(persuasion_doc)
aliceentities = more_bag_of_words_ent(alice_doc)
persuasionentities = more_bag_of_words_ent(persuasion_doc)
alicepos = more_bag_of_words_pos(alice_doc)
persuasionpos = more_bag_of_words_pos(persuasion_doc)

#Finding unique key words

# Combine bags to create a set of unique words/pharases
common_words = list(set(alicewords + persuasionwords))
common_phrases = list(set(alicephrases + persuasionphrases))
common_entities = list(set(aliceentities + persuasionentities))
common_pos = list(set(alicepos + persuasionpos))
unique_words_alice_notinPersuasion=set(alicewords)- set(persuasionwords)
unique_words_persuasion_notinAlice=set(persuasionwords)-set(alicewords)

In [252]:

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences=sentences, common_words=common_words,
                 common_entities=common_entities, common_pos=common_pos, common_phrases=common_phrases):
    
    # Scaffold the data frame and initialize counts to zero.
    columns=common_words+common_entities+common_pos+common_phrases
    df = pd.DataFrame(columns=columns)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, columns] = 0
     
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        phrases = [ph.text
                 for ph in sentence.noun_chunks
                 if (' ' in str(ph)) and not ('*' in str(ph)) and ph.text in common_phrases]
    
    
        entities = [e.label_
                 for e in sentence.ents
                 if (e.label_ in common_entities
                 )]
        pos = [token.pos_
                 for token in sentence
                 if (token.pos_ in common_pos
                 )]
                    # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        #Populate the row with phrase counts.
        for phrase in phrases:
            df.loc[i, phrase] += 1
            
        # Populate the row with phrase counts.
        for entity in entities:
            df.loc[i, entity] += 1
            
        for token in pos:
            df.loc[i, token] += 1
        # This counter is just to make sure the kernel didn't hang.
        if i % 100 == 0:
            print("Processing row {}".format(i))
                        
    return df

word_counts = bow_features()
word_counts.head()

Processing row 0
Processing row 100
Processing row 200
Processing row 300
Processing row 400


,Nay,passage,deputation,inhabit,sense,nation,perfectly,steal,elegance,soothing,...,his first marriage,a much safer place,such the feelings,a toil,our wealthy naval commanders,a VERY good opportunity,a man,a state,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll


In [253]:
word_counts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 443 entries, 0 to 442
Columns: 2787 entries, Nay to text_source
dtypes: object(2787)
memory usage: 9.4+ MB


In [254]:
Y = word_counts['text_source']
X = word_counts.drop(['text_sentence','text_source'], 1)

In [303]:
def model(mod, X=X, Y=Y):
    #Data split to Train and test
    global X_train, X_test, y_train, y_test
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=1)
    mo=mod
    mo.fit(X_train, y_train)
    print ("Model Train score is %0.2f" % (mo.score(X_train, y_train)) )
    print ("Model Test score is %0.2f" % (mo.score(X_test, y_test)))
    return mo

In [256]:
model(ensemble.RandomForestClassifier())

Model Train score is 0.98
Model Test score is 0.82


/Users/Sri/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [257]:
model(ensemble.GradientBoostingClassifier())

Model Train score is 0.98
Model Test score is 0.86


In [258]:
model(LogisticRegression(solver='lbfgs'))

Model Train score is 0.98
Model Test score is 0.87


In [250]:
model(SVC(gamma='auto', kernel='linear', C=1e9 ))

Model Train score is 1.00
Model Test score is 0.83


## When added common pos, common phrases and  common entities the accuracy increased from 83% to 87% i.e a 4% increase.

In [ ]:
#With just pos alone but a different way of taking in the parts of speech together for each sentence as one feature.

In [259]:

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences=sentences):
    df=pd.DataFrame()
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
       
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
      
        #pos = [token.pos_ for token in sentence]
        
        dep = [token.dep_ for token in sentence]
        
        #df['pos'+ str(i)]=0
        #df['pos'+ str(i)][(i-1):i] = 1
        
        df['dep'+ str(i)]=0
        df['dep'+ str(i)][(i-1):i] = 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
                        
    return df

spacy_df = bow_features()
spacy_df.head()

/Users/Sri/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400


,text_sentence,text_source,dep0,dep1,dep2,dep3,dep4,dep5,dep6,dep7,...,dep433,dep434,dep435,dep436,dep437,dep438,dep439,dep440,dep441,dep442
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"(So, she, was, considering, in, her, own, mind...",Carroll,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"(Oh, dear, !)",Carroll,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"(Oh, dear, !)",Carroll,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [260]:
Y = spacy_df['text_source']
X = spacy_df.drop(['text_sentence','text_source'], 1)

In [261]:
model(LogisticRegression(solver='lbfgs'))

Model Train score is 0.98
Model Test score is 0.87


### With just pos or dep or combined the result is 87% i.e alone they can equal all the other features combined in accuracy. 

This is very interesting phenomenon, which needs to discussed and analysed in detail later.

A different approach one can take is to see if taking unique key words in each text alice and persuasion and see that is predictive

In [321]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords)]
    
# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

#Finding unique key words
unique_words_alice_notinPersuasion=set(alicewords)- set(persuasionwords)
unique_words_persuasion_notinAlice=set(persuasionwords)-set(alicewords)

In [322]:

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences=sentences,
                 unique_words_alice_notinPersuasion=unique_words_alice_notinPersuasion,
                unique_words_persuasion_notinAlice=unique_words_persuasion_notinAlice):
    columns=list(unique_words_persuasion_notinAlice)+list(unique_words_alice_notinPersuasion)
    df=pd.DataFrame(columns=columns)
    
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, columns] = 0
       
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        words = [token.lemma_ for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in (unique_words_persuasion_notinAlice
                     or unique_words_alice_notinPersuasion)
                 )]
        
        for word in words:
                df.loc[i, word] += 1
       
       
        pos = [token.pos_ for token in sentence]
        
        dep = [token.dep_ for token in sentence]
        
        df['pos'+ str(i)]=0
        df['pos'+ str(i)][i:(i+1)] = 1
        
        df['dep'+ str(i)]=0
        df['dep'+ str(i)][i:(i+1)] = 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
                        
    return df

spacy_df = bow_features()
spacy_df.head()

/Users/Sri/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Sri/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400


,Q,S,2,K,T,`,I,G,z,9,...,pos438,dep438,pos439,dep439,pos440,dep440,pos441,dep441,pos442,dep442
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [323]:
Y = spacy_df['text_source']
X = spacy_df.drop(['text_sentence','text_source'], 1)

In [324]:
mo=model(LogisticRegression(solver='lbfgs'))

Model Train score is 0.99
Model Test score is 0.88


In [238]:
model(ensemble.GradientBoostingClassifier())

Model Train score is 0.98
Model Test score is 0.86


Here we see that with using unique key words alone or in combination with other pos and dep all give 88% accuracy.

The next attempt is to combine al the features so far done.

In [402]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]

def more_bag_of_words(doc):
    
    # Filter out punctuation and stop words.
    phrases = [chunk.text for chunk in doc.noun_chunks
               if (' ' in str(chunk)) and not ('*' in str(chunk))]
    
    # Return the most common words.
    return [item[0] for item in Counter(phrases).most_common(2000)]    

def more_bag_of_words_ent(doc):
    
    # Filter out punctuation and stop words.
    entities = [chunk.label_ for chunk in doc.ents]
    #if (' ' in str(chunk)) and not ('*' in str(chunk))
    # Return the most common words.
    return [item[0] for item in Counter(entities).most_common(2000)]  

def more_bag_of_words_pos(doc):
    
    sentences=doc.sents
    
    pos = [token.pos_ for sent in sentences for token in sent]
    #if (' ' in str(chunk)) and not ('*' in str(chunk))
    # Return the most common words.
    #return [item[0] for item in Counter(pos).most_common(2000)]  
    return pos
    
# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)
alicephrases = more_bag_of_words(alice_doc)
persuasionphrases = more_bag_of_words(persuasion_doc)
aliceentities = more_bag_of_words_ent(alice_doc)
persuasionentities = more_bag_of_words_ent(persuasion_doc)
alicepos = more_bag_of_words_pos(alice_doc)
persuasionpos = more_bag_of_words_pos(persuasion_doc)

#Finding unique key words

# Combine bags to create a set of unique words/pharases
common_words = list(set(alicewords + persuasionwords))
common_phrases = list(set(alicephrases + persuasionphrases))
common_entities = list(set(aliceentities + persuasionentities))
common_pos = list(set(alicepos + persuasionpos))
unique_words_alice_notinPersuasion=set(alicewords)- set(persuasionwords)
unique_words_persuasion_notinAlice=set(persuasionwords)-set(alicewords)

In [403]:

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences=sentences, common_words=common_words,
                 common_entities=common_entities, common_pos=common_pos, common_phrases=common_phrases,
                unique_words_alice_notinPersuasion=unique_words_alice_notinPersuasion,
                unique_words_persuasion_notinAlice=unique_words_persuasion_notinAlice):
    
    # Scaffold the data frame and initialize counts to zero.
    columns=common_words+common_entities+common_pos+common_phrases+list(unique_words_alice_notinPersuasion)+list(unique_words_persuasion_notinAlice)
    
    df = pd.DataFrame(columns=columns)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, columns] = 0
     
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        uniq_words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in (unique_words_persuasion_notinAlice
                     or unique_words_alice_notinPersuasion))]
        phrases = [ph.text
                 for ph in sentence.noun_chunks
                 if (' ' in str(ph)) and not ('*' in str(ph)) and ph.text in common_phrases]
    
    
        entities = [e.label_
                 for e in sentence.ents
                 if (e.label_ in common_entities
                 )]
        pos = [token.pos_
                 for token in sentence
                 if (token.pos_ in common_pos
                 )]
                    # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # Populate the row with phrase counts.
        for phrase in phrases:
            df.loc[i, phrase] += 1
            
        # Populate the row with phrase counts.
        for entity in entities:
            df.loc[i, entity] += 1
            
        for token in pos:
            df.loc[i, token] += 1
        for word in uniq_words:
            df.loc[i, word] += 1
     
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 100 == 0:
            print("Processing row {}".format(i))
        
    return df

word_counts = bow_features()
word_counts.head()

Processing row 0
Processing row 100
Processing row 200
Processing row 300
Processing row 400


,Nay,passage,deputation,inhabit,sense,nation,perfectly,steal,elegance,soothing,...,neighbourhood,economy,embarrassment,compassion,scarcely,preside,well,Russell,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll


In [404]:
Y = word_counts['text_source']
X = word_counts.drop(['text_sentence','text_source'], 1)

In [405]:
model(LogisticRegression(solver='lbfgs'))

Model Train score is 0.99
Model Test score is 0.88


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Challenge 1

In [ ]:
With all the features the accuracy acheivable till now is 87%. Let us how we can apply this model to other texts.

In [281]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [377]:
# Clean the Edgeworth-parents data.
edge = gutenberg.raw('edgeworth-parents.txt')
edge = re.sub('[A-Z]* [A-Z]*\.', '', edge)
edge = text_cleaner(edge[:int(len(alice))])
print(edge[:100])

Near the ruins of the castle of Rossmore, in Ireland, is a small cabin, in which there once lived a 


In [378]:
# Parse our cleaned data.
edge_doc = nlp(edge)

In [379]:
# Group into sentences.
edge_sents = [[sent, "Edgeworth"] for sent in edge_doc.sents]

In [406]:
# Build a new Bag of Words data frame for Edge word count
sentences = pd.DataFrame(alice_sents+edge_sents)
sentences.head()
sentences.shape

(239, 2)

In [407]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]

def more_bag_of_words(doc):
    
    # Filter out punctuation and stop words.
    phrases = [chunk.text for chunk in doc.noun_chunks
               if (' ' in str(chunk)) and not ('*' in str(chunk))]
    
    # Return the most common words.
    return [item[0] for item in Counter(phrases).most_common(2000)]    

def more_bag_of_words_ent(doc):
    
    # Filter out punctuation and stop words.
    entities = [chunk.label_ for chunk in doc.ents]
    #if (' ' in str(chunk)) and not ('*' in str(chunk))
    # Return the most common words.
    return [item[0] for item in Counter(entities).most_common(2000)]  

def more_bag_of_words_pos(doc):
    
    sentences=doc.sents
    
    pos = [token.pos_ for sent in sentences for token in sent]
    #if (' ' in str(chunk)) and not ('*' in str(chunk))
    # Return the most common words.
    return [item[0] for item in Counter(pos).most_common(2000)]  

    
# Set up the bags.
alicewords = bag_of_words(alice_doc)
edgewords = bag_of_words(edge_doc)
alicephrases = more_bag_of_words(alice_doc)
edgephrases = more_bag_of_words(edge_doc)
aliceentities = more_bag_of_words_ent(alice_doc)
edgeentities = more_bag_of_words_ent(edge_doc)
alicepos = more_bag_of_words_pos(alice_doc)
edgepos = more_bag_of_words_pos(edge_doc)

#Finding unique key words

# Combine bags to create a set of unique words/pharases
common_words = list(set(alicewords + edgewords))
common_phrases = list(set(alicephrases + edgephrases))
common_entities = list(set(aliceentities + edgeentities))
common_pos = list(set(alicepos + edgepos))
unique_words_alice_notinEdge=set(alicewords)- set(edgewords)
unique_words_Edge_notinAlice=set(edgewords)-set(alicewords)

In [408]:

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences=sentences, common_words=common_words,
                 common_entities=common_entities, common_pos=common_pos, common_phrases=common_phrases,
                unique_words_alice_notinEdge=unique_words_alice_notinEdge,
                unique_words_Edge_notinAlice=unique_words_Edge_notinAlice):
    
    # Scaffold the data frame and initialize counts to zero.
    columns=common_words+common_entities+common_pos+common_phrases+list(unique_words_alice_notinEdge)+list(unique_words_Edge_notinAlice)
    
    df = pd.DataFrame(columns=columns)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, columns] = 0
     
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        uniq_words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in (unique_words_Edge_notinAlice
                     or unique_words_alice_notinEdge))]
        phrases = [ph.text
                 for ph in sentence.noun_chunks
                 if (' ' in str(ph)) and not ('*' in str(ph)) and ph.text in common_phrases]
    
    
        entities = [e.label_
                 for e in sentence.ents
                 if (e.label_ in common_entities
                 )]
        pos = [token.pos_
                 for token in sentence
                 if (token.pos_ in common_pos
                 )]
                    # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # Populate the row with phrase counts.
        for phrase in phrases:
            df.loc[i, phrase] += 1
            
        # Populate the row with phrase counts.
        for entity in entities:
            df.loc[i, entity] += 1
            
        for token in pos:
            df.loc[i, token] += 1
        for word in uniq_words:
            df.loc[i, word] += 1
        pos = [token.pos_ for token in sentence]
        
        dep = [token.dep_ for token in sentence]
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 100 == 0:
            print("Processing row {}".format(i))
        
    return df

word_counts = bow_features()
word_counts.head()

Processing row 0
Processing row 100
Processing row 200


,dear,mind,grow,hint,passage,seek,large,bethink,employ,stock,...,fourpence,rush,softly,nature,bleach,eld,fine,guinea,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll


In [409]:
Y = word_counts['text_source']
X = word_counts.drop(['text_sentence','text_source'], 1)

In [411]:
model(LogisticRegression(solver='lbfgs'))

Model Train score is 0.99
Model Test score is 0.88


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

The model does well with 88%(same as got before) when used to train another text document edge along with alice txt. It has got the same accuracy as alice and persuasion.